<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/nvidia_neo4j_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet langchain-nvidia-ai-endpoints langchain-community neo4j langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.7/293.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.8/379.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.1 MB/s eta 0:00:00


In [2]:
import os
from typing import Any, Dict, List, Optional, Tuple, Type

from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import \
    format_to_openai_function_messages
from langchain.agents.output_parsers.openai_tools import \
    OpenAIToolsAgentOutputParser
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun,
                                         CallbackManagerForToolRun)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import AIMessage, HumanMessage
from langchain.tools import BaseTool
from langchain.tools.render import format_tool_to_openai_function

# Build a knowledge graph-based agent with Llama 3.1, NVIDIA NIM, and LangChain
## Leverage Llama-3.1 native function-calling capabilities to retrieve structured data from a knowledge graph to power your RAG applications

While most people focus on RAG over unstructured text, such as company documents or documentation, I am pretty bullish on retrieval systems over structured information, particularly knowledge graphs. There has been a lot of excitement about GraphRAG, specifically Microsoft's implementation. However, in their implementation, the input data is unstructured text in the form of documents, which is transformed into a knowledge graph using a Large Language Model (LLM).

In this blog post, we will show how to implement a retriever over a knowledge graph containing structured information from FDA Adverse Event Reporting System (FAERS), which offers the information about drug adverse events. If you have ever tinkered with knowledge graphs and retrieval, your first thought might be to use an LLM to generate database queries to retrieve relevant information from a knowledge graph to answer a given question. However, database query generation using LLMs is still evolving and may not yet offer the most consistent or robust solution. So, what are the viable alternatives at the moment?

In my opinion, the best solution at the moment is the so-called dynamic query generation. Rather than relying entirely on an LLM to generate the complete query, this method employs a logic layer that deterministically generates a database query from predefined input parameters.his solution can be implemented using an LLM with function-calling support. The advantage of using a function-calling feature lies in the ability to define to an LLM how it should prepare a structured input to a function. This approach ensures that the query generation process is controlled and consistent while allowing for user input flexibility.

![image](https://cdn-images-1.medium.com/max/1200/1*5N_tHHvD-rY9ZP_nnANDMw.png)

The image illustrates a  process of a understanding a user's question to retrieve specific information. The flow involves three main steps:
A user asks a question about common side effects of the drug Lyrica for people under 35 years old.

The LLM decides which function to call and the parameters needed. In this example, it chose a function named "side_effects" with parameters including the drug "Lyrica" and a maximum age of 35.

The identified function and parameters are used to deterministically and dynamically generate a database query (Cypher) statement to retrieve relevant information.

Function-calling support is vital for advanced LLM use cases, such as allowing LLMs to use multiple retrievers based on user intent or building multi-agent flows. I have written some articles using commercial LLMs with native function-calling support. However, in this blog post, we will use Llama-3.1, a superior open-source LLM with native function-calling support that was released just recently.
## Setting up the knowledge graph
We will use Neo4j, which is a native graph database to store the adverse event information. You can set up a free cloud Sandbox project that comes with pre-populated FAERS by following [this link](https://sandbox.neo4j.com/?usecase=healthcare-analytics).
The instantiated database instance has a graph with the following schema.

![image](https://cdn-images-1.medium.com/max/800/1*hM90ShEOOWhbQ-6_OcrWYg.png)

The schema centers on the Case node, which links various aspects of a drug safety report, including the drugs involved, reactions experienced, outcomes, and therapies prescribed. Each drug is characterized by whether it is primary, secondary, concomitant, or interacting. Cases are also associated with information about the manufacturer, the age group of the patient, and the source of the report. This schema allows for tracking and analyzing the relationships between drugs, their reactions, and outcomes in a structured manner.

We'll start by creating a connection to the database by instantiating a Neo4jGraph object.

In [3]:
os.environ["NEO4J_URI"] = "bolt://18.206.157.187:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "elevation-reservist-thousands"

graph = Neo4jGraph(refresh_schema=False)

## Setting up the LLM environment
There are many options to host open-source LLMs like the Llama-3.1. In this blog post, we will use the [NVIDIA API catalog](https://www.nvidia.com/en-us/ai/#referrer=ai-subdomain?ncid=ref-inpa-144886), which provides NVIDIA NIM inference microservices and supports function calling for llama 3.1 models. When you create an account you get 1000 tokens, which is more than enough to complete this blog post. You'll need to create an API key and copy it to the notebook.

In [4]:
os.environ["NVIDIA_API_KEY"] = "nvapi-"
llm = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")

We'll be using the **llama-3.1–70b** as the 8b version has some hiccups with optional parameters in function definitions.
The nice thing about NVIDIA NIM microservices is that you can easily host them locally if you have security or other concerns, so it's really easily swappable and you only need to add a url parameter to the LLM configuration.

In [5]:
# connect to an local NIM running at localhost:8000,
# specifying a specific model
# llm = ChatNVIDIA(
#  base_url="http://localhost:8000/v1",
#  model="meta/llama-3.1-70b-instruct"
# )

In [6]:
result = llm.invoke("How to use LLMs in combination with Graph Databases? Be concise!")
print(result.content)

Here's a concise guide on using Large Language Models (LLMs) with Graph Databases:

**Why combine LLMs with Graph Databases?**

* Enhance semantic search and querying capabilities
* Leverage graph-based relationships for more accurate and informative results
* Augment graph data with text-based insights and entity recognition

**How to combine LLMs with Graph Databases:**

1. **Text preprocessing**: Use LLMs to preprocess text data (e.g., NLP, entity recognition, sentiment analysis) before ingesting it into the graph database.
2. **Graph embedding**: Use LLMs to generate graph embeddings, which can be used to represent nodes and edges in the graph database.
3. **Query augmentation**: Use LLMs to augment graph queries with natural language understanding, enabling more expressive and flexible querying.
4. **Graph-based knowledge graph completion**: Use LLMs to predict missing relationships and entities in the graph database, enhancing knowledge graph completion.
5. **Entity disambiguatio

## Tool definition
In this example, we will configure a single tool with four optional parameters. Based on those parameters, we will construct a corresponding Cypher statement that will be used to retrieve the relevant information from the knowledge graph.

First we will define some utility functions

In [7]:
graph.query(
    "CREATE FULLTEXT INDEX drug IF NOT EXISTS FOR (d:Drug) ON EACH [d.name];"
)
graph.query(
    "CREATE FULLTEXT INDEX manufacturer IF NOT EXISTS FOR (d:Manufacturer) ON EACH [d.manufacturerName];"
)

[]

In [8]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2) to each word, then combines them using the AND
    operator. Useful for mapping movies and people from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


candidate_query = """
CALL db.index.fulltext.queryNodes($index, $fulltextQuery, {limit: $limit})
YIELD node
RETURN coalesce(node.manufacturerName, node.name) AS candidate,
       labels(node)[0] AS label
"""


def get_candidates(input: str, type: str, limit: int = 3) -> List[Dict[str, str]]:
    """
    Retrieve a list of candidate entities from database based on the input string.

    This function queries the Neo4j database using a full-text search. It takes the
    input string, generates a full-text query, and executes this query against the
    specified index in the database. The function returns a list of candidates
    matching the query, with each candidate being a dictionary containing their name
    (or title) and label (either 'Person' or 'Movie').
    """
    ft_query = generate_full_text_query(input)
    candidates = graph.query(
        candidate_query, {"fulltextQuery": ft_query, "index": type, "limit": limit}
    )
    return candidates

In [9]:
get_candidates("voriconazol", "drug")

[{'candidate': 'VORICONAZOLE', 'label': 'Drug'}]

Specifically, our tool will be able to identify the most frequent side effects based on input drug, age, and the drug manufacturer.

In [10]:
@tool
def get_side_effects(
    drug: Optional[str] = Field(
        description="disease mentioned in the question. Return None if no mentioned."
    ),
    min_age: Optional[int] = Field(
        description="Minimum age of the patient. Return None if no mentioned."
    ),
    max_age: Optional[int] = Field(
        description="Maximum age of the patient. Return None if no mentioned."
    ),
    manufacturer: Optional[str] = Field(
        description="manufacturer of the drug. Return None if no mentioned."
    ),
):
    """Useful for when you need to find common side effects."""
    params = {}
    filters = []
    side_effects_base_query = """
    MATCH (c:Case)-[:HAS_REACTION]->(r:Reaction), (c)-[:IS_PRIMARY_SUSPECT]->(d:Drug)
    """
    if drug and isinstance(drug, str):
        candidate_drugs = [el["candidate"] for el in get_candidates(drug, "drug")]
        if not candidate_drugs:
            return "The mentioned drug was not found"
        filters.append("d.name IN $drugs")
        params["drugs"] = candidate_drugs

    if min_age and isinstance(min_age, int):
        filters.append("c.age > $min_age ")
        params["min_age"] = min_age
    if max_age and isinstance(max_age, int):
        filters.append("c.age < $max_age ")
        params["max_age"] = max_age
    if manufacturer and isinstance(manufacturer, str):
        candidate_manufacturers = [
            el["candidate"] for el in get_candidates(manufacturer, "manufacturer")
        ]
        if not candidate_manufacturers:
            return "The mentioned manufacturer was not found"
        filters.append(
            "EXISTS {(c)<-[:REGISTERED]-(:Manufacturer {manufacturerName: $manufacturer})}"
        )
        params["manufacturer"] = candidate_manufacturers[0]

    if filters:
        side_effects_base_query += " WHERE "
        side_effects_base_query += " AND ".join(filters)
    side_effects_base_query += """
    RETURN d.name AS drug, r.description AS side_effect, count(*) AS count
    ORDER BY count DESC
    LIMIT 10
    """
    print(f"Using parameters: {params}")
    data = graph.query(side_effects_base_query, params=params)
    return data

The get_side_effectsfunction is designed to retrieve common side effects of drugs from a knowledge graph using specified search criteria. It accepts optional parameters for drug name, patient age range, and drug manufacturer to customize the search. Each parameter has a description that is passed to an LLM along with the function description, enabling the LLM to understand how to use them. The function then constructs a dynamic Cypher query based on the provided inputs, executes this query against the knowledge graph, and returns the resulting side effects data.

Let's test the function.

In [11]:
get_side_effects("lyrica")

Using parameters: {'drugs': ['LYRICA', 'LYRICA CR']}


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'drug': 'LYRICA', 'side_effect': 'Pain', 'count': 32},
 {'drug': 'LYRICA', 'side_effect': 'Fall', 'count': 21},
 {'drug': 'LYRICA',
  'side_effect': 'Intentional product use issue',
  'count': 20},
 {'drug': 'LYRICA', 'side_effect': 'Insomnia', 'count': 19},
 {'drug': 'LYRICA', 'side_effect': 'Feeling abnormal', 'count': 18},
 {'drug': 'LYRICA', 'side_effect': 'Drug ineffective', 'count': 18},
 {'drug': 'LYRICA', 'side_effect': 'Memory impairment', 'count': 17},
 {'drug': 'LYRICA', 'side_effect': 'Withdrawal syndrome', 'count': 17},
 {'drug': 'LYRICA', 'side_effect': 'Malaise', 'count': 16},
 {'drug': 'LYRICA', 'side_effect': 'Intentional product misuse', 'count': 15}]

Our tool first mapped the "lyrica" drug mentioned in the question to "['LYRICA', 'LYRICA CR']" values in the knowledge graph and then executed corresponding Cypher statement to find the most frequent side effects.
## Graph-based LLM Agent
The only thing left to do is configure an LLM agent that can use the defined tool to answer questions about the drug's side effects.

![image](https://cdn-images-1.medium.com/max/800/1*5Q4y5emOAhR7kw2L7rpeaw.png)

The image depicts a user interacting with a Llama-3.1 agent to inquire about drug side effects. The agent accesses a side effects tool that retrieves information from a knowledge graph to provide the user with the relevant data.

We'll start by defining the prompt template.

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that finds information about common side effects. "
            "If tools require follow up questions, "
            "make sure to ask the user for clarification. Make sure to include any "
            "available options that need to be clarified in the follow up questions "
            "Do only the things the user specifically requested. ",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

The prompt template includes the system message, optional chat history, and user input. The agent_scratchpad is reserved for the LLM, as it sometimes needs to perform multiple steps to answer the question, like executing and retrieving information from tools.

The LangChain library makes it straightforward to add tools to the LLM by using the bind_tools method.

In [13]:
tools = [get_side_effects]
llm_with_tools = llm.bind_tools(tools=tools)

agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"])
        if x.get("chat_history")
        else [],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)


# Add typing for input
class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


class Output(BaseModel):
    output: Any


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput, output_type=Output
)

The agent processes input through a series of transformations and handlers that format the chat history, apply the LLM with the bound tools, and parse the output. Finally, the agent is set up with an executor that manages the execution flow, specifies input and output types, and includes verbosity settings for detailed logging during execution.

Let's now test the agent.

In [14]:
agent_executor.invoke(
    {
        "input": "What are the most common side effects when using lyrica for people below 35 years old?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `get_side_effects` with `{'drug': 'lyrica', 'max_age': 35}`


Using parameters: {'drugs': ['LYRICA', 'LYRICA CR'], 'max_age': 35}
[{'drug': 'LYRICA', 'side_effect': 'Sinusitis', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Hypoacusis', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Fall', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Brain injury', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Amnesia', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Off label use', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Impaired quality of life', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Somnolence', 'count': 1}, {'drug': 'LYRICA', 'side_effect': 'Drug ineffective for unapproved indication', 'count': 1}]Based on the provided data, the most common side effects of Lyrica for people below 35 years old are sinusitis, hypoacusis, fall, brain injury, amnesia, off-label use, impaired quality of life, somnolence, and drug ineffective fo

{'input': 'What are the most common side effects when using lyrica for people below 35 years old?',
 'output': 'Based on the provided data, the most common side effects of Lyrica for people below 35 years old are sinusitis, hypoacusis, fall, brain injury, amnesia, off-label use, impaired quality of life, somnolence, and drug ineffective for unapproved indication.'}

In [15]:
agent_executor.invoke(
    {
        "input": "What are the most common side effects when for drugs manufactured by acadia?"
    }
)




> Entering new AgentExecutor chain...

Invoking: `get_side_effects` with `{'manufacturer': 'acadia'}`


Using parameters: {'manufacturer': 'ACADIA PHARMACEUTICALS'}
[{'drug': 'NUPLAZID', 'side_effect': 'Hallucination', 'count': 13}, {'drug': 'NUPLAZID', 'side_effect': 'Confusional state', 'count': 7}, {'drug': 'NUPLAZID', 'side_effect': 'Fall', 'count': 6}, {'drug': 'NUPLAZID', 'side_effect': 'Delusion', 'count': 5}, {'drug': 'NUPLAZID', 'side_effect': 'Gait disturbance', 'count': 5}, {'drug': 'NUPLAZID', 'side_effect': 'Fatigue', 'count': 4}, {'drug': 'NUPLAZID', 'side_effect': 'Abnormal behaviour', 'count': 3}, {'drug': 'NUPLAZID', 'side_effect': 'Product dose omission issue', 'count': 3}, {'drug': 'NUPLAZID', 'side_effect': 'Agitation', 'count': 3}, {'drug': 'NUPLAZID', 'side_effect': 'Death', 'count': 3}]The most common side effects for drugs manufactured by Acadia are: Hallucination, Confusional state, Fall, Delusion, Gait disturbance, Fatigue, Abnormal behaviour, Product dose o

{'input': 'What are the most common side effects when for drugs manufactured by acadia?',
 'output': 'The most common side effects for drugs manufactured by Acadia are: Hallucination, Confusional state, Fall, Delusion, Gait disturbance, Fatigue, Abnormal behaviour, Product dose omission issue, Agitation, and Death.'}

The LLM identified it needs to use the get_side_effects function with appropriate arguments. The function then dynamically generates a Cypher statement, fetches the relevant information, and returns it to the LLM to generate the final answer.
## Summary
Function calling capabilities are a powerful addition to open-source models like Llama 3.1, enabling more structured and controlled interactions with external data sources and tools. Beyond just querying unstructured documents, graph-based agents offer exciting possibilities for interacting with knowledge graphs and structured data. The ease of hosting these models using platforms like [NVIDIA NIM microservices](https://www.nvidia.com/en-us/ai/#referrer=ai-subdomain?ncid=ref-inpa-144886) makes them increasingly accessible.